# 3. 比例ハザードモデルの解釈
本jupyterファイルでは、比例ハザードモデルによる結果の解釈を行う手法とコードを示す。

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 日本語フォントを設定
font = {'family': 'IPAexGothic'}
mpl.rc('font', **font)

%matplotlib inline

In [8]:
# データセットの読み込み
from sksurv.datasets import load_veterans_lung_cancer

x, y = load_veterans_lung_cancer()
x.head()

,Age_in_years,Celltype,Karnofsky_score,Months_from_Diagnosis,Prior_therapy,Treatment
0,69.0,'squamous',60.0,7.0,'no','standard'
1,64.0,'squamous',70.0,5.0,'yes','standard'
2,38.0,'squamous',60.0,3.0,'no','standard'
3,63.0,'squamous',60.0,9.0,'yes','standard'
4,65.0,'squamous',70.0,11.0,'yes','standard'


In [9]:
y

array([( True,   72.), ( True,  411.), ( True,  228.), ( True,  126.),
       ( True,  118.), ( True,   10.), ( True,   82.), ( True,  110.),
       ( True,  314.), (False,  100.), ( True,   42.), ( True,    8.),
       ( True,  144.), (False,   25.), ( True,   11.), ( True,   30.),
       ( True,  384.), ( True,    4.), ( True,   54.), ( True,   13.),
       (False,  123.), (False,   97.), ( True,  153.), ( True,   59.),
       ( True,  117.), ( True,   16.), ( True,  151.), ( True,   22.),
       ( True,   56.), ( True,   21.), ( True,   18.), ( True,  139.),
       ( True,   20.), ( True,   31.), ( True,   52.), ( True,  287.),
       ( True,   18.), ( True,   51.), ( True,  122.), ( True,   27.),
       ( True,   54.), ( True,    7.), ( True,   63.), ( True,  392.),
       ( True,   10.), ( True,    8.), ( True,   92.), ( True,   35.),
       ( True,  117.), ( True,  132.), ( True,   12.), ( True,  162.),
       ( True,    3.), ( True,   95.), ( True,  177.), ( True,  162.),
      

観測期間中に死亡した場合は`Status`が`True`になり、`False`の場合は右側打切りである。

In [15]:
# 学習済みモデルの読み込み
from sklearn.externals import joblib
from sksurv.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split

# 共変量
use_cols = [
    'Age_in_years',
    'Celltype',
    'Karnofsky_score',
    'Months_from_Diagnosis',
    'Prior_therapy',
    'Treatment'
]

# カテゴリー変数をダミー変数へ変換
encoder_x = OneHotEncoder().fit_transform(x[use_cols])

data_x = encoder_x
data_y = y

# 交差検証(ホールドアウト法)
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, train_size=0.7, random_state=0)

# モデルをロードする
FILENAME = 'cox_ph_model_v1.sav'
cox_ph_model = joblib.load(FILENAME)
result = cox_ph_model.score(x_test, y_test)
print(result)
print(data_x.columns.values)

0.69618696187
['Age_in_years' "Celltype='large'" "Celltype='smallcell'"
 "Celltype='squamous'" 'Karnofsky_score' 'Months_from_Diagnosis'
 "Prior_therapy='yes'" "Treatment='test'"]


/Users/taiyou/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/sksurv/column.py:147: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  cat = pandas.Categorical.from_array(pandas.Series(data))


In [21]:
coef_df = DataFrame([data_x.columns, cox_ph_model.coef_]).T
coef_df.rename(columns={0: '変数', 1: '回帰係数'}, inplace=True)
coef_df

,変数,回帰係数
0,Age_in_years,-0.0178705
1,Celltype='large',-0.842767
2,Celltype='smallcell',0.00444637
3,Celltype='squamous',-1.22398
4,Karnofsky_score,-0.0381951
5,Months_from_Diagnosis,-0.00415532
6,Prior_therapy='yes',-0.164954
7,Treatment='test',0.154291


---
## 3.1 比例ハザードモデルの解釈
### 比例ハザード性
1つの共変量を含む回帰モデルを考える。比例ハザードモデルにおけるハザード関数は、
$$
h\left(t,x,\beta\right) = h_{0}\left(t\right)e^{x\beta}
$$
であり、一般化線形モデルの枠組みでいうそのリンク関数は、自然対数変換する対数ハザード関数である。つまり
$$
\ln {h\left(t,x,\beta\right)} = \ln {h_{0}\left(t\right)} + x\beta \qquad (1)
$$
となる。$x=a$から$x=b$までの変化の対数ハザード関数の差は、
$$
\left( \ln {h_{0}\left(t\right)} + a\beta \right) - \left( \ln {h_{0}\left(t\right)} + b\beta \right) = a\beta - b\beta\\
= (a-b)\beta \qquad (2)
$$
となる。<font color="red">ここで、比例ハザード性の仮定より上式の対数ハザードの差が時間に依存しないことに注意すること</font>。

#### <font color="blue">ハザード比</font>
そして、対数ハザードを指数変換して共変量の変化に対する影響を評価するハザード比を定義する。
$$
HR\left( t, x=a,b, \beta\right) = exp {\left[ \left( \ln {h_{0}\left(t\right)} a\beta \right) - \left( \ln {h_{0}\left(t\right)} b\beta \right)  \right]}\\
= \frac {h\left(t,a,\beta\right)}{h\left(t,b,\beta\right)}\\
= e^{(a-b)\beta} \qquad (3)
$$

このハザード比は、**<font color="blue">全体の期間にわたる生存状況の比較手段であり</font>**、(3)式から得られたハザード比は、**観察期間中はいつでも**$x=a$の属性の1単位期間あたりの死亡率(イベント発生率)が$x=b$の属性の$e^{(a-b)\beta}$倍であることを意味する。

### 名義尺度共変量
まず、1つの二値または名義尺度共変量の回帰係数の解釈について考える。
共変量が、$a=0,b=1$もしくは$a=+1,b=-1$のような二値変数の値であるとき、ハザード比の推定値は、
$$
\hat {HR}\left(t,a,b,\hat {\beta}\right) = e^{(a-b)\hat {\beta}} \qquad (5)
$$
となる。ハザード比の$100(1-\alpha)$%信頼区間の区間推定は、
$$
exp {\left[ (a-b)\hat {\beta} \pm \left| a-b \right| \cdot z_{1-\alpha/2} \cdot \hat {SE}\left(\hat {\beta}\right) \right]} \qquad (6)
$$
となる。

 - $\hat {SE}\left(\beta\right)$: 標準誤差

### 連続尺度共変量
連続尺度共変量の$c$単位の変化に対応する対数ハザード関数の変化を$a = x + c$および$b = x$として、(2)式および(3)式を用いると、対数ハザード関数の変化は、
$$
\left\{ \ln {h_{0}\left(t\right)} + (x+c)\beta \right\} - \left\{ \ln {h_{0}\left(t\right)} + x\beta \right\} = (x+c)\beta - x\beta\\
= c\beta
$$
となる。ハザード比は、
$$
\hat {HR}\left(c\right) = e^{c\hat {\beta}}
$$
として推定され、ハザード比の$100(1-\alpha)$%信頼区間の区間推定は、
$$
exp {\left[ c\hat {\beta} \pm z_{1-\alpha/2} \cdot \left| c\right| \cdot \hat {SE}\left(\hat {\beta}\right) \right]}
$$
である。

In [25]:
# ハザード比の追加
HAZARD_RATIO  = {
    'Age_in_years': 10,
    "Celltype='large'": 1,
    "Celltype='smallcell'": 1,
    "Celltype='squamous'": 1,
    'Karnofsky_score': 10,
    'Months_from_Diagnosis': 3,
    "Prior_therapy='yes'": 1,
    "Treatment='test'": 1
}

def add_point_estimation_of_hazard_ratio(row):
    """
    ハザード比の点推定の値を返す
    """
    variable, beta = row
    if variable in HAZARD_RATIO:
        n_th = HAZARD_RATIO[variable]
    else:
        n_th = 1
    return np.exp(n_th * beta)


def add_interval_estimation_of_hazard_ratio(row, alpha = 0.05):
    """
    ハザード比の区間推定の値を返す
    """
    return low_estimated_value, high_estimated_value

coef_df['ハザード比'] = coef_df[['変数', '回帰係数']].apply(add_point_estimation_of_hazard_ratio, axis=1)
# coef_df['ハザード比の区間推定]
coef_df

,変数,回帰係数,ハザード比
0,Age_in_years,-0.0178705,0.836352
1,Celltype='large',-0.842767,0.430518
2,Celltype='smallcell',0.00444637,1.004456
3,Celltype='squamous',-1.22398,0.294059
4,Karnofsky_score,-0.0381951,0.682529
5,Months_from_Diagnosis,-0.00415532,0.987611
6,Prior_therapy='yes',-0.164954,0.847933
7,Treatment='test',0.154291,1.166830


### 多変量モデル
リスク因子または治療変数$d$と、他の変数$x$を考える。ここで、共変量$x$は結果変数と有意に関連していると仮定する。

共変量$x$は次のモデルの当てはめを考える。

 - $x$を含むモデル
 - (1)交絡因子: $d$を含むが、$x$は含まないモデル
 - (2)関連性の効果修飾因子: $d$と$x$の両方を含むモデル
 - (2)交絡因子または効果修飾因子のどちらでもない: $d,x$とそれらの交互作用項$x \times d$を含むモデル

事例の前に、交絡と交互作用に対する概念的理解と視覚的把握を促すため、2変数の設定を考える。

主たるリスク因子$d$は2水準(0=なし、1=あり)であり、主な解析の目的は$d$のハザード比を推定することであると仮定する。<br>

$d$のみを含む比例ハザードモデルは、対数ハザード関数
$$
\ln {h_{0}\left(t\right)} + d \theta_{1}
$$
をもつ。$d$の2水準での対数ハザードの差は、
$$
\left( \ln {h_{0}\left(t\right)} + 1\theta_{1} \right) - \left( \ln {h_{0}\left(t\right)} + 0\theta_{1} \right) = \theta_{1}
$$
となる。

$d$と$x$の両方を含む２番目のモデルにおける対数ハザード関数は、
$$
\ln {h_{0}\left(t\right)} + d \beta_{1} + x\beta_{2}
$$
となる。上式から、調整された対数ハザードの差は、
$$
\left( \ln {h_{0}\left(t\right)} + 1\beta_{1} + x\beta_{2} \right) - \left( \ln {h_{0}\left(t\right)} + 0\beta_{1} + x\beta_{2} \right) = \beta_{1} + (x - x)\beta_{2}\\
= \beta_{1}
$$
となる。